In [ ]:
import pandas as pd
import numpy as np
train_df= pd.read_csv("../input/santander-customer-transaction-prediction/train.csv")
test_df = pd.read_csv('../input/santander-customer-transaction-prediction/test.csv')
sample_submission = pd.read_csv('../input/santander-customer-transaction-prediction/sample_submission.csv')

In [ ]:
features = train_df.columns.values[2:202]

In [ ]:
from sklearn.model_selection import train_test_split
import lightgbm
from sklearn.metrics import roc_auc_score


In [ ]:

X = train_df.loc[:,features]
y = train_df.loc[:,'target']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

train_data = lightgbm.Dataset(X_train, label=y_train)
valid_data = lightgbm.Dataset(X_valid, label=y_valid)

parameters = {'objective': 'binary',
              'metric': 'auc',
              'is_unbalance': 'true',
              'boosting': 'gbdt',
              'num_leaves': 63,
              'feature_fraction': 0.5,
              'bagging_fraction': 0.5,
              'bagging_freq': 20,
              'learning_rate': 0.01,
              'verbose': -1
             }
model_lgbm = lightgbm.train(parameters,
                            train_data,
                            valid_sets=valid_data,
                            num_boost_round=5000,
                            early_stopping_rounds=50)


In [ ]:
y_train_pred = model_lgbm.predict(X_train)
y_valid_pred = model_lgbm.predict(X_valid)

print("AUC Train: {:.4f}\nAUC Valid: {:.4f}".format(roc_auc_score(y_train, y_train_pred),
                                                    roc_auc_score(y_valid, y_valid_pred)))

In [ ]:
y_test=model_lgbm.predict(test_df[features])

In [ ]:
test_df['target']=y_test#np.rint(y_test).astype(int)
submit=test_df[['ID_code','target']]
submit.to_csv('submission.csv',index=False)

In [ ]:
#submit.target.value_counts()

LGBM

In [ ]:
#from lightgbm import LGBMClassifier
#model = LGBMClassifier()

Logistic Regression

In [ ]:
#from sklearn.linear_model import LogisticRegression
#model = LogisticRegression(solver='lbfgs', max_iter=5000)

OverSampling: It doesn't yield better score compared to undersampling.

In [ ]:
#from imblearn.over_sampling import SMOTE
#oversample = SMOTE()
#X, y = oversample.fit_resample(train_df[features], train_df.target)

Undersampling

In [ ]:
#df0=train_df[train_df.target==0].sample(20098)
#df1=train_df[train_df.target==1]
#df=pd.concat([df0,df1],axis=0)
#shuffle
#train=df.sample(frac=1)
#X=train[features]
#y=train.target

**Notes**
* A basic logistic regression initially had problems while training which was fixed by increasing the number of max_iter value from default 100 to 5000. A score of 0.63 was achieved.

* Downsampling data to match the value counts of '1' in target column massively increased the score to 0.777. It also saved a lot of training time by a factor of ~10.

* On oversampling using SMOTE, the score didn't increase compared to downsampling. It had added negative effect of massively increasing the training time. 

* LGBMClassifier increased the score by only a tiny amount (negligible)

* Training lightgbm with custom defined run_lgb function that has its own parameters defined improved the score to 0.8; which can be considered a pretty good improvement